In [5]:
import pandas as pd
df = pd.read_json (r'covert_starcode_Q&A_t=0.5.json')
#pd.DataFrame([my_dict])
df.to_csv (r'covert_starcode_Q&A_t=0.5.txt', index = True)

In [3]:
import pandas as pd
df_text = pd.read_csv('Te2Query.csv')
eg = df_text.sample(n=1000, random_state=3)
input_text = eg['Questions'].to_list()
input_labels = eg['query'].to_list()
eg

,Unnamed: 0,Questions,id,query
3276,3276,Give me all the patients who got vaccines on ...,20103276,"POST_scripts/1{""script"":{""lang"":""mustache"",""so..."
1409,1409,Give me all the patients whose report was comp...,7101409,"POST_scripts/1{""script"":{""lang"":""mustache"",""so..."
7172,7172,Which is the most common cataracts for patients.,28307172,"POST_scripts/3{""script"":{""lang"":""mustache"",""so..."
9319,9319,What is the number of records that the vaccin...,33109319,"POST_scripts/1{""script"":{""lang"":""mustache"",""so..."
11467,11467,Give me all the patients who got INFLUENZA (SE...,43311467,"POST_scripts/3{""script"":{""lang"":""mustache"",""so..."
...,...,...,...,...
7332,7332,Give me all the patients who was allergic to pvc,29307332,"POST_scripts/3{""script"":{""lang"":""mustache"",""so..."
10466,10466,How many GLAXOSMITHKLINE BIOLOGICALS vaccine ...,37110466,"POST_scripts/1{""script"":{""lang"":""mustache"",""so..."
1109,1109,How many patients are 100.0 years old.,3201109,"POST_scripts/2{""script"":{""lang"":""mustache"",""so..."
7771,7771,What is the number of vaccine recipients that...,29307771,"POST_scripts/3{""script"":{""lang"":""mustache"",""so..."


In [4]:
eg['query'].to_csv (r'egq.txt', index = True)

In [8]:
pip install datasketch

/bin/bash: /home/wenlong/anaconda3/envs/llama/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Obtaining dependency information for datasketch from https://files.pythonhosted.org/packages/81/71/fb0c28eff49fc0d725782f6dcf4ba2f71c52b6e6e9575179df3802b19d90/datasketch-1.6.4-py3-none-any.whl.metadata
Using cached datasketch-1.6.4-py3-none-any.whl (88 kB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
# -*- coding: utf-8 -*-

# 正则包
import re
# 自然语言处理包
import jieba
import jieba.analyse
# html 包
import html
# 数据集处理包
from datasketch import MinHash


class MinHashSimilarity(object):
    """
    MinHash
    """
    def __init__(self, content_x1, content_y2):
        self.s1 = content_x1
        self.s2 = content_y2

    @staticmethod
    def extract_keyword(content):  # 提取关键词
        # 正则过滤 html 标签
        re_exp = re.compile(r'(<style>.*?</style>)|(<[^>]+>)', re.S)
        content = re_exp.sub(' ', content)
        # html 转义符实体化
        content = html.unescape(content)
        # 切割
        seg = [i for i in jieba.cut(content, cut_all=True) if i != '']
        # 提取关键词
        keywords = jieba.analyse.extract_tags("|".join(seg), topK=200, withWeight=False)
        return keywords

    def main(self):
        # 去除停用词
        #jieba.analyse.set_stop_words('stopwords.txt')

        # MinHash计算
        m1, m2 = MinHash(), MinHash()
        # 提取关键词
        s1 = self.extract_keyword(self.s1)
        s2 = self.extract_keyword(self.s2)

        for data in s1:
            m1.update(data.encode('utf8'))
        for data in s2:
            m2.update(data.encode('utf8'))

        return m1.jaccard(m2)


# 测试
if __name__ == '__main__':
    with open('covert_starcode_Q&A_t=0.5.txt', 'r') as x, open('eg.txt', 'r') as y:
        content_x = x.read()
        content_y = y.read()
        similarity = MinHashSimilarity(content_x, content_y)
        similarity = similarity.main()
        print('相似度: %.2f%%' % (similarity*100))

相似度: 21.88%
